In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.regularizers import l2
import tensorflow_datasets as tfds

# Завантаження даних Fashion MNIST за допомогою TensorFlow Datasets
(ds_train, ds_test), ds_info = tfds.load(
    'fashion_mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Нормалізація даних до діапазону [0, 1]
def normalize_img(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

ds_train = ds_train.map(normalize_img)
ds_test = ds_test.map(normalize_img)

# Побудова моделі
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),  # Згортковий шар
    MaxPooling2D((2, 2)),  # Пулінг шар
    Flatten(),  # Вирівнювання вхідних зображень
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),  # Збільшимо кількість нейронів та додамо L2 регуляризацію
    BatchNormalization(),  # Додамо шар пакетної нормалізації
    Dropout(0.5),  # Дропаут для регуляризації
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),  # Додамо ще один прихований шар
    Dropout(0.3),  # Зменшимо дропаут для цього шару
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),  # Додамо ще один прихований шар
    Dense(10, activation='softmax')  # Вихідний шар з 10 класами (класифікація)
])

# Компіляція моделі
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=SparseCategoricalCrossentropy(),
              metrics=[SparseCategoricalAccuracy()])

# Навчання моделі
history = model.fit(ds_train.batch(128), epochs=30,  # Збільшимо кількість епох
                    validation_data=ds_test.batch(128), verbose=2,
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])  # Додамо зупинку раннього навчання

# Оцінка точності на тестовому наборі
test_loss, test_acc = model.evaluate(ds_test.batch(128), verbose=2)
print(f"Test accuracy: {test_acc}")


ModuleNotFoundError: No module named 'resource'